# Install python binance api

In [4]:
!pip install python-binance -q

#Import required libraries

In [40]:
from binance.client import Client
import pandas as pd

# Grab information

In [41]:
# We don't need any specific api key and password to get long short ratio data
api_key = ''
api_secret = ''

# Connect to the api
client = Client(api_key, api_secret)

In [54]:
# Grab exchange info to get all future pairs
exchange_info = client.futures_exchange_info()
pairs = pd.DataFrame.from_dict(exchange_info['symbols'])['pair']

ls_ratio = []
spreads = []
funding = []
# Get average long short ratio during a period of time
period = '5m'
# Get n last periods
nLastPeriods = 1
for pair in pairs:
  funding.append(float(client.futures_funding_rate(symbol=pair, limit=nLastPeriods)[0]['fundingRate']))
  ls_ratio.append(float(client.futures_global_longshort_ratio(symbol=pair, period=period, limit=nLastPeriods)[0]['longShortRatio']))
  try:
     spreads.append(float(client.futures_symbol_ticker(symbol=pair)['price']) - float(client.get_symbol_ticker(symbol=pair)['price']))
  except: 
    spreads.append(float("nan"))

In [55]:
# Aggregate data
table = pd.DataFrame.from_dict(exchange_info['symbols'])[['pair']]
table['global long short ratio'] = ls_ratio
table['spreads'] = spreads
table['funding'] = funding

# Export data

In [56]:
# Sort values by ascending order
table = table.sort_values(by=['global long short ratio'])

# Write down values
table.to_excel('lsratio.xlsx')
table.to_html('lsratio.html', classes='table table-striped')

# You can retrieve the html and xlsx files in the files tab (folder icon on the left panel)